In [ ]:
%%capture
# install unsloth for faster inference
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
#login to huggingface
from google.colab import userdata
secret_hf = userdata.get('HUGGINGFACE_TOKEN')
!huggingface-cli login --token $secret_hf

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Koni99/mistral-7b-RAFT-unsloth2-merged", # Choose the RAFT fine-tuned model
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.4
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
# Load the test dataset
dataset = load_dataset("Koni99/ISOT_test_final")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# load Liar testset
from datasets import load_dataset

# If the dataset is gated/private, make sure you have run huggingface-cli login
dataset = load_dataset("Koni99/LIAR_test")

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
#load F3 testset
from datasets import load_dataset

# If the dataset is gated/private, make sure you have run huggingface-cli login
dataset = load_dataset("Koni99/F3_postgpt_testset")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset['train'][280]

{'Unnamed: 0': 280,
 'uuid': 'b7f41c61-9c5f-44ab-99ff-3261e1090f51',
 'text': 'On 6 November 2017, the U.S. Department of Justice (DOJ) announced it was dropping its case against\xa0Desiree Fairooz, 61, a Code Pink activist arrested in January for laughing out loud during the Senate confirmation hearing of Attorney General nominee Jeff Sessions. The Bluemont, Virginia resident was convicted in May on misdemeanor charges of disorderly conduct and parading or demonstrating on capitol grounds stemming from the incident, but the verdict was overturned by a Superior Court judge on the grounds that laughter alone wasn\'t sufficient cause for conviction. She was to have been re-tried on 13 November. The January 2017 arrest wasn\'t the first for the\xa0longtime Code Pink member, who was jailed for\xa0disorderly conduct and assaulting a police officer in October 2007 after covering\xa0her hands with fake blood\xa0and disrupting a Congressional hearing attended by then-U.S. Secretary of State\xa

In [ ]:
# test inference
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(dataset["train"]["prompt"][100], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 500, use_cache = True)
model_output = tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
print(model_output)

['<s><s> [INST] You are an expert AI fact checker trained to detect fake news. Is this article real or fake news? Analyze the given text in detail as a fact checker would. Explain your reasoning step-by-step.  Check for misleading information, false claims, biased language. If the news article is real, respond with 1 for \'True\', if fake, respond with 0 for \'False\'.  Structure your answer as a JSON object with the two keys \'rationale\' for your analytic reasoning and \'prediction\' for your final prediction.  The values for \'prediction\' are binary. Either 1 for a real article or 0 for fake news. In \'rationale\' you provide a string with your detailed reasoning process that led you to the decision you took.   Remember if the article contains misleading information, false claims, biased language or any other hints of disinformation your prediction should be 0 for fake news, else 1 for true news.   To summarize the JSON object looks like this: \'rationale\': <your_analytic_reasonin

In [ ]:
extracted_model_output = model_output[0].split("[/INST]")[1].strip()

In [ ]:
extracted_model_output

'The article provides detailed evidence and analysis to debunk the claims made by Russian propagandists regarding the situation in Ukraine. It explains that the video in question was not filmed in Ukraine in 2022, but rather in 2013 in England. The article also refutes the claims of crisis actors and miscaptioned footage being shared by news outlets. The language used in the article is factual and objective, presenting evidence to support the conclusion that the video and related claims are false. Therefore, the article is real news.</s>'

In [ ]:
def generated_response(input_article):
  inputs = tokenizer(input_article, return_tensors = "pt").to("cuda")
  outputs = model.generate(**inputs, max_new_tokens = 500, use_cache = True)
  model_output = tokenizer.batch_decode(outputs)
  extracted_model_output = model_output[0].split("[/INST]")[1].strip()
  return extracted_model_output

In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
torch.cuda.empty_cache()

In [ ]:
torch.cuda.empty_cache()
generated_response(dataset["train"]["prompt"][1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'The article provides a detailed analysis of the claim that U.S. Rep. Marjorie Taylor Greene repeated a 1928 statement by Adolf Hitler about Christianity. It explains that while Hitler did make such a statement, there is no evidence that Greene ever repeated it. The article also includes direct quotes from Greene\'s interviews and speeches, showing that she referred to herself as a "Christian nationalist" but did not repeat Hitler\'s exact words. The article also mentions the backlash Greene faced for her remarks. Overall, the article presents a factual analysis of the claim and provides evidence to support its conclusion that the meme is false.</s>'

In [ ]:
# run the fine-tuned model on the test dataset
from tqdm import tqdm
results = []
for i in tqdm(range(len(dataset['train']))):
  results.append(generated_response(dataset["train"]["prompt"][i]))

 87%|████████▋ | 288/330 [31:26<03:54,  5.59s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
100%|██████████| 330/330 [35:39<00:00,  6.48s/it]


In [ ]:
import re
def extract_prediction(text):
    model_prediction = "Check manually"
    match = re.search(r'["\']?[pP]rediction["\']?\s*:\s*["\']?(\d)["\']?', text)
    if match:
        model_prediction = int(match.group(1))
    else:
        if re.search(r'\bfake news\b\.',text) and re.search(r'\breal news\b\.', text):
            model_prediction = "Both detected check manually "
        elif re.search(r'\bfake news\b\.', text):
            if not re.search(r'\b no[t]? fake news\b\.', text):
                model_prediction = 0
        elif re.search(r'\breal news\b\.', text):
            if not re.search(r'\b no[t]? real news\b\.', text):
                model_prediction = 1
    return model_prediction

In [ ]:
extract_prediction(results[0])

1

In [ ]:
def extend_data(raw_results):
  predictions = []
  raw = []
  for i in range(len(results)):
    predictions.append(extract_prediction(results[i]))
    raw.append(raw_results[i])
  return predictions, raw

In [ ]:
import pandas as pd
def save_results(dataframe,raw_results, filename):
    predictions, raw = extend_data(raw_results)
    pd_df = pd.DataFrame(dataframe)
    extended_dataframe = pd_df.assign(model_prediction = predictions, raw_model_response = raw)
    # Save the DataFrame as a CSV file
    extended_dataframe.to_csv(filename, index=True)

In [ ]:
# save the results as .csv file
save_results(dataset["train"],results,"unsloth2_f3_new.csv")